# Assignment Solutions

In [27]:
# imports
from torchvision.datasets import MNIST
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# Use MNIST dataset variable
mnist_data_train = MNIST('/', train=True, download=True)
mnist_data_test = MNIST('/', train=False, download=True)

In [29]:
# Filter samples for classes 1 and 7
def filter_dataset(dataset):
    indices = (dataset.targets == 1) | (dataset.targets == 7)
    images = dataset.data[indices]
    labels = dataset.targets[indices]
    return images, labels

X_train, y_train = filter_dataset(mnist_data_train)
X_test, y_test = filter_dataset(mnist_data_test)

# Assign labels +1 to class 1 and -1 to class 7
y_train = np.where(y_train == 1, 1, -1)
y_test = np.where(y_test == 1, 1, -1)

# Vectorize the data
X_train = X_train.reshape(-1, 28 * 28) / 255.0  # Normalize pixel values to range [0, 1]
X_test = X_test.reshape(-1, 28 * 28) / 255.0

# Split the data into training and testing sets (80-20 split)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Calculate the weight vector using the Moore-Penrose pseudoinverse
X_train_with_bias = np.c_[np.ones((X_train.shape[0], 1)), X_train]
weights = np.linalg.pinv(X_train_with_bias) @ y_train

# Add bias term (a column of 1s) to X_val for the intercept term
X_val_with_bias = np.c_[np.ones((X_val.shape[0], 1)), X_val]

# Predict on the validation set
y_pred = np.sign(X_val_with_bias @ weights)

# Calculate accuracy and confusion matrix for the validation set
accuracy = np.mean(y_pred == y_val)
conf_matrix = np.zeros((2, 2))
for i in range(len(y_val)):
    conf_matrix[y_val[i] >= 0, y_pred[i] >= 0] += 1

print("Validation Accuracy:", accuracy)
print("Validation Confusion Matrix:")
print(conf_matrix)

# Add bias term (a column of 1s) to X_test for the intercept term
X_test_with_bias = np.c_[np.ones((X_test.shape[0], 1)), X_test]

# Predict on the test set
y_pred_test = np.sign(X_test_with_bias @ weights)

# Calculate accuracy and confusion matrix for the test set
accuracy_test = np.mean(y_pred_test == y_test)
conf_matrix_test = np.zeros((2, 2))
for i in range(len(y_test)):
    conf_matrix_test[y_test[i] >= 0, y_pred_test[i] >= 0] += 1

print("Test Accuracy:", accuracy_test)
print("Test Confusion Matrix:")
print(conf_matrix_test)


Validation Accuracy: 0.9888547271329746
Validation Confusion Matrix:
[[1365. 1365.]
 [1365. 1365.]]
Test Accuracy: 0.9875173370319001
Test Confusion Matrix:
[[1132. 1132.]
 [1132. 1132.]]


In [30]:
# Question 3
import numpy as np

# Step 1: Generate random vectors p and q
p = np.random.uniform(-10, 10, 100)
q = np.random.uniform(-10, 10, 100)

# Step 2: Generate target variable b based on specified conditions
b = np.where(p * q > 1, 1, -1)

# Step 3: Define basis functions
def f0(p, q):
    return np.ones_like(p)

def f1(p, q):
    return p

def f2(p, q):
    return q

def f3(p, q):
    return p ** 2

def f4(p, q):
    return q ** 2

def f5(p, q):
    return p * q

# Step 4: Create the matrix A using the basis functions
A = np.column_stack([f0(p, q), f1(p, q), f2(p, q), f3(p, q), f4(p, q), f5(p, q)])

# Step 5: Use least squares to find the coefficient vector x
x = np.linalg.lstsq(A, b, rcond=None)[0]

# Step 6: Compute the predicted values
y_pred = np.dot(A, x)
# print(y_pred)
# print(b)

# Step 7: Compute the mean squared error
mse = np.mean((b - y_pred) ** 2)
print(mse)

0.41047617238671635


In [31]:
import numpy as np
import pandas as pd

# Read data from CSV
data = pd.read_csv('ques_9_data.csv')

# Convert data to numpy array
data = data.values

# Divide data into three segments based on the constraints
data_seg_1 = data[(data[:, 0] >= 0) & (data[:, 0] <= 1)]
data_seg_2 = data[(data[:, 0] > 1) & (data[:, 0] <= 2)]
data_seg_3 = data[(data[:, 0] > 2) & (data[:, 0] <= 3)]

# Define functions for each segment
def polynomial_f1(params, x):
    return np.polyval(params, x)

def polynomial_f2(params, x):
    return np.polyval(params, x)

def polynomial_f3(params, x):
    return np.polyval(params, x)

# Function to calculate residuals for each segment
def residuals(params, x, y, polynomial):
    return polynomial(params, x) - y

# Perform least squares fitting for each segment
def least_squares_fit(segment, degree):
    x = segment[:, 0]
    y = segment[:, 1]
    
    # Vandermonde matrix
    X = np.vander(x, degree + 1)
    
    # Calculate parameters using the normal equation
    params = np.linalg.lstsq(X, y, rcond=None)[0]
    # convert params to list
    params = params.tolist()
    return params

# Fit polynomials to the segments
params_f1 = least_squares_fit(data_seg_1, 2)  # Degree 2 for segment 1
params_f2 = least_squares_fit(data_seg_2, 3)  # Degree 3 for segment 2
params_f3 = least_squares_fit(data_seg_3, 2)  # Degree 2 for segment 3

# Print the optimized parameters for each segment till 4 decimal places
print('f1(x) = {:.2f} + {:.2f}x + {:.2f}x^2'.format(params_f1[0], params_f1[1], params_f1[2]))
print('f2(x) = {:.2f} + {:.2f}x + {:.2f}x^2 + {:.2f}x^3'.format(params_f2[0], params_f2[1], params_f2[2], params_f2[3]))
print('f3(x) = {:.2f} + {:.2f}x + {:.2f}x^2'.format(params_f3[0], params_f3[1], params_f3[2]))

f1(x) = 1.00 + -3.00x + 4.00x^2
f2(x) = 3.00 + -2.00x + -0.00x^2 + 1.00x^3
f3(x) = 1.00 + 24.00x + -35.00x^2
